# Model Selection and evaluation

This notebook contains the methods for evaluating the performance of combinations of models.

In [1]:
### Imports

import numpy as np
import pandas as pd
import os
pd.options.mode.chained_assignment = None
# for dirname, _, filenames in os.walk('../input/prostate-cancer-grade-assessment/train_images'):
#     for filename in filenames:
#         if filename == 'c40fab4c4c3658f40cc9a73c8602d8d0.tiff':
#             print(os.path.join(dirname, filename))

%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *

import os
from PIL import Image
import pandas as pd
import numpy as np
import cv2
import PIL
import random
import openslide
import skimage.io
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [2]:
### DataFrames

path = Path('/kaggle/input/pandas-to-jpeg-with-tiles-sample')

pd.set_option('mode.chained_assignment', None)

train = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
train[['primary Gleason', 'secondary Gleason']] = train.gleason_score.str.split('+',expand=True)
display(train.head())

train['id'] = train['image_id'] + '.jpeg'
train_isup = train[['id', 'isup_grade']]
train_primary = train[['id', 'primary Gleason']]
train_secondary = train[['id', 'secondary Gleason']]

train_isup['isup_grade'] = train_isup['isup_grade'].astype(int)


train_isup['grade_1'] = pd.Series([1 if x >= 1 else 0 for x in train_isup['isup_grade']], index=train_isup.index)
train_isup['grade_2'] = pd.Series([1 if x >= 2 else 0 for x in train_isup['isup_grade']], index=train_isup.index)
train_isup['grade_3'] = pd.Series([1 if x >= 3 else 0 for x in train_isup['isup_grade']], index=train_isup.index)
train_isup['grade_4'] = pd.Series([1 if x >= 4 else 0 for x in train_isup['isup_grade']], index=train_isup.index)
train_isup['grade_5'] = pd.Series([1 if x >= 5 else 0 for x in train_isup['isup_grade']], index=train_isup.index)

train_bin = train_isup[['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5']]

test_df = pd.read_csv('../input/prostate-cancer-grade-assessment/test.csv')
sample_submission = pd.read_csv('../input/prostate-cancer-grade-assessment/sample_submission.csv')

,image_id,data_provider,isup_grade,gleason_score,primary Gleason,secondary Gleason
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,0,0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,0,0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,4,4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,4,4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,0,0


<a id="part-one"></a>
# Create Models

Object data is a databunch object which resizes all input images to size 448, data_512 resizes all input images to size 512.

[Return to Table of Contents](#ToC)

In [3]:
# WINDOW_SIZE = 200 ### Values for merge large dataset
# STRIDE = 100
# K = 25


data = ImageDataBunch.from_df(path=Path('/kaggle/input/pandas-to-jpeg-with-tiles-sample'),
                              df=train_bin.loc[:1000], #train_isup (the actual target), train_primary, train_secondary
                              valid_pct=0.25,
                              label_col=['grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'],
                              size=448,
                              bs=8,
                              ds_tfms=get_transforms()
        ).normalize(imagenet_stats)

data_512 = ImageDataBunch.from_df(path=Path('/kaggle/input/pandas-to-jpeg-with-tiles-sample'),
                              df=train_bin.loc[:1000], #train_isup (the actual target), train_primary, train_secondary
                              valid_pct=0.25,
                              label_col=['grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'],
                              size=512,
                              bs=8,
                              ds_tfms=get_transforms()
        ).normalize(imagenet_stats)

data_686 = ImageDataBunch.from_df(path=Path('/kaggle/input/pandas-to-jpeg-with-tiles-sample'),
                              df=train_bin.loc[:1000], #train_isup (the actual target), train_primary, train_secondary
                              valid_pct=0.25,
                              label_col=['grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'],
                              size=686,
                              bs=8,
                              ds_tfms=get_transforms()
        ).normalize(imagenet_stats)

data_784 = ImageDataBunch.from_df(path=Path('/kaggle/input/pandas-to-jpeg-with-tiles-sample'),
                              df=train_bin.loc[:1000], #train_isup (the actual target), train_primary, train_secondary
                              valid_pct=0.25,
                              label_col=['grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'],
                              size=784,
                              bs=4,
                              ds_tfms=get_transforms()
        ).normalize(imagenet_stats)

### Model objects:
The cells below instantiate each model that is used for inference. To add new models simply copy one of the cells below and modify the object name, model path, Databunch, and load name.

In [4]:
learners = [] #this variable will hold a list of models that will be iterated through at inference time
learner_dfs = [] #this variable will hold a list of dfs that will be iterated through to store predictions from each model

In [5]:
learn_1 = cnn_learner(data, models.densenet161, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/densenet161-with-tiles-progressive-resize-448/')
learn_1.model_dir = Model_Path
learn_1.load('checkpoint-5');
Model_Path = Path('/kaggle/working/')
learn_1.model_dir = Model_Path

learners += [learn_1]

learn_df_1 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_1]

In [6]:
learn_2 = cnn_learner(data, models.vgg16_bn, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/with-tiles-progressive-resize-448/')
learn_2.model_dir = Model_Path
learn_2.load('checkpoint-5')
Model_Path = Path('/kaggle/working/')
learn_2.model_dir = Model_Path

learners += [learn_2]

learn_df_2 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_2]

In [7]:
learn_3 = cnn_learner(data_512, models.vgg16_bn, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/panda-models/')
learn_3.model_dir = Model_Path
learn_3.load('checkpoint-3')
Model_Path = Path('/kaggle/working/')
learn_3.model_dir = Model_Path

learners += [learn_3]

learn_df_3 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_3]

In [8]:
learn_4 = cnn_learner(data, models.densenet161, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/densenet161-with-tiles-progressive-resize-448/')
learn_4.model_dir = Model_Path
learn_4.load('checkpoint-6')
Model_Path = Path('/kaggle/working/')
learn_4.model_dir = Model_Path

learners += [learn_4]

learn_df_4 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_4]

In [9]:
learn_5 = cnn_learner(data, models.vgg16_bn, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/with-tiles-progressive-resize-448/')
learn_5.model_dir = Model_Path
learn_5.load('checkpoint-4')
Model_Path = Path('/kaggle/working/')
learn_5.model_dir = Model_Path

learners += [learn_5]

learn_df_5 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_5]

In [10]:
learn_6 = cnn_learner(data, models.resnet50, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/resnet-with-tiles-progressive-resize')
learn_6.model_dir = Model_Path
learn_6.load('checkpoint-6')
Model_Path = Path('/kaggle/working/')
learn_6.model_dir = Model_Path

learners += [learn_6]

learn_df_6 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_6]

In [11]:
learn_7 = cnn_learner(data_686, models.densenet169, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/densenet169size686/')
learn_7.model_dir = Model_Path
learn_7.load('dense169-checkpoint-4')
Model_Path = Path('/kaggle/working/')
learn_7.model_dir = Model_Path

learners += [learn_7]

learn_df_7 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_7]

In [12]:
learn_8 = cnn_learner(data_686, models.densenet169, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/densenet169size686/')
learn_8.model_dir = Model_Path
learn_8.load('dense169-checkpoint-5')
Model_Path = Path('/kaggle/working/')
learn_8.model_dir = Model_Path

learners += [learn_8]

learn_df_8 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_8]

In [13]:
learn_9 = cnn_learner(data_686, models.resnet50, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/densenet169size686/')
learn_9.model_dir = Model_Path
learn_9.load('resnet50-checkpoint-5')
Model_Path = Path('/kaggle/working/')
learn_9.model_dir = Model_Path

learners += [learn_9]

learn_df_9 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_9]

In [14]:
learn_10 = cnn_learner(data_784, models.densenet201, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/pandas-size-784-models/')
learn_10.model_dir = Model_Path
learn_10.load('dense201-checkpoint-5')
Model_Path = Path('/kaggle/working/')
learn_10.model_dir = Model_Path

learners += [learn_10]

learn_df_10 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_10]

In [15]:
learn_11 = cnn_learner(data_784, models.densenet201, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/pandas-size-784-models/')
learn_11.model_dir = Model_Path
learn_11.load('dense201-checkpoint-4')
Model_Path = Path('/kaggle/working/')
learn_11.model_dir = Model_Path

learners += [learn_11]

learn_df_11 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_11]

In [16]:
learn_12 = cnn_learner(data_784, models.resnet101, metrics=accuracy , pretrained=False) #resnet101, densenet161, vgg16_bn
Model_Path = Path('/kaggle/input/pandas-size-784-models/')
learn_12.model_dir = Model_Path
learn_12.load('resnet101-checkpoint-2')
Model_Path = Path('/kaggle/working/')
learn_12.model_dir = Model_Path

learners += [learn_12]

learn_df_12 = pd.DataFrame(columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5'])
learner_dfs += [learn_df_12]

In [17]:
# This cell is just to confirm all models where added succesfully

for l in learners:
    print('i')

i
i
i
i
i
i
i
i
i
i
i
i


# Quadratic Weighted Kappa Function

The code for this metric comes from https://www.kaggle.com/reighns/understanding-the-quadratic-weighted-kappa by Kaggle user reigHns

In [18]:
from sklearn.metrics import confusion_matrix

def quadratic_kappa(actuals, preds, N=6):
    """This function calculates the Quadratic Kappa Metric used for Evaluation in the PetFinder competition
    at Kaggle. It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values 
    of adoption rating."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/(N-1)**2)
    
    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1
                         
    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

# Evaluation

This section evaluates combinations of algorythms to see which subsets are the best performing in a blend.

### Inference Function
The models were trained to classify images into an array of 5 binary categories. For example a class two gleason score would be [1,1,0,0,0], and a class four would be [1,1,1,1,0]. The idea is to get the model to see these classes as ordinal without treating the problem as a classic regression. For averaging the results of multiple models we get the probabilities assigned to each value in the array from each model, average them then round them for each element, and then sum them. By averaging the results of multiple models you can get better generalization.
 
For example we might have one model which sees an image and assigns the probabilities of positive cases as [0.9, 0.9, 0.6, 0.1, 0.1] (this means that the model sees a class one and a class two as likely but not the others). The next model could assign the probabilities to [0.9, 0.8, 0.3, 0.1, 0.05]. By taking the average of the two we get a score of [0.9, 0.85, 0.45, 0.1, 0.075], which rounded becomes [1, 1, 0, 0, 0], then summed equals 2, so the diagnosis would be a class two. In this case model one would have given a score of class 3 and model two would have given a score of class 2, model one's prediction however wasn't as certain as model twos (its probability for a class three was hovering by the threshold and not firmly on one side or the other).

In [19]:
%%time
## Here we store the predictions for each model in its own dataframe

cases = 1000


def inference_testing(data_dir, df, filler=-1):
    z = 0
    for i in df['image_id']:
            z+=1
            img = open_image(data_dir+i+'.jpeg')
            count = 0
            for l in learners:
                pred_class,pred_idx,outputs = l.predict(img)
                learner_dfs[count] = learner_dfs[count].append(pd.DataFrame([[i, float(outputs[0]), float(outputs[1]), float(outputs[2]), float(outputs[3]), float(outputs[4])]], columns=['id', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5']))
                count+=1
    return df, z

data_dir = '/kaggle/input/pandas-to-jpeg-with-tiles-sample/'
t, z = inference_testing(data_dir, train[['image_id', 'isup_grade']].head(cases), filler=-1)
t['actual'] = train_isup['isup_grade'].head(cases)

CPU times: user 8h 34min 9s, sys: 30min 9s, total: 9h 4min 19s
Wall time: 4h 32min 23s


In [20]:
#%%time

## This function will take a list of indexs and calculate the Quadratic Weighted Kappa
## for the combination of models at the index locations in the model list



df_out = t[['image_id', 'isup_grade', 'actual']].head(cases)
df_out['isup_grade'] = [np.nan for x in df_out['isup_grade']]

def get_QWK(df, idx_list=[0,1]):
    for i in df['image_id']:
        s_1 = []
        s_2 = []
        s_3 = []
        s_4 = []
        s_5 = []
        s_all = [s_1, s_2, s_3, s_4, s_5]
        idx = 0
        for l in learner_dfs:
            if idx in idx_list:
                _t = l.loc[l.id == i]
                s_1 += [_t.grade_1[0]]
                s_2 += [_t.grade_2[0]]
                s_3 += [_t.grade_3[0]]
                s_4 += [_t.grade_4[0]]
                s_5 += [_t.grade_5[0]]
                #print(idx, end=' ')
            idx+=1
        count = 0
        for s in s_all:
            s_all[count] = round(np.mean(s))
            count+=1
        _ = sum(s_all)
        df_out['isup_grade'].loc[df_out.image_id == i] = _
    return quadratic_kappa(df_out['isup_grade'].astype(int), df_out['actual'].astype(int))


In [21]:
%%time
# This section runs through every 3 model combination of model in the model list
# This can be useful to determin which combinations of models produce the best performance


from itertools import combinations

num = len(learner_dfs)
for e in itertools.combinations(range(num), 3):
    score = get_QWK(t, list(e))
    if score > .83:
        print('Quadratic Weighted Kappa for models',list(e), 'is: ', score)

Quadratic Weighted Kappa for models [0, 1, 2] is:  0.8381223085029903
Quadratic Weighted Kappa for models [0, 1, 3] is:  0.8405111467286166
Quadratic Weighted Kappa for models [0, 1, 4] is:  0.8338113001470598
Quadratic Weighted Kappa for models [0, 1, 5] is:  0.8342731411628314
Quadratic Weighted Kappa for models [0, 1, 6] is:  0.8484000546279362
Quadratic Weighted Kappa for models [0, 1, 7] is:  0.8571470230288596
Quadratic Weighted Kappa for models [0, 1, 8] is:  0.8451715785559053
Quadratic Weighted Kappa for models [0, 1, 9] is:  0.8358618998806473
Quadratic Weighted Kappa for models [0, 1, 10] is:  0.8391962906913594
Quadratic Weighted Kappa for models [0, 1, 11] is:  0.8444866116601064
Quadratic Weighted Kappa for models [0, 2, 5] is:  0.8334538528537987
Quadratic Weighted Kappa for models [0, 2, 6] is:  0.8452480693641109
Quadratic Weighted Kappa for models [0, 2, 7] is:  0.8539855191646291
Quadratic Weighted Kappa for models [0, 2, 8] is:  0.8388686720136467
Quadratic Weighted

In [22]:
#gives individual model performance
for e in range(num):
    print('Quadratic Weighted Kappa for model ',e, 'is: ', get_QWK(t, [e]))

Quadratic Weighted Kappa for model  0 is:  0.841024904232432
Quadratic Weighted Kappa for model  1 is:  0.8197517103602225
Quadratic Weighted Kappa for model  2 is:  0.8058825810993668
Quadratic Weighted Kappa for model  3 is:  0.8242974824626582
Quadratic Weighted Kappa for model  4 is:  0.8020865460075303
Quadratic Weighted Kappa for model  5 is:  0.8124875280493487
Quadratic Weighted Kappa for model  6 is:  0.8438450611222167
Quadratic Weighted Kappa for model  7 is:  0.8535342227099743
Quadratic Weighted Kappa for model  8 is:  0.8312278591003274
Quadratic Weighted Kappa for model  9 is:  0.7974310575019566
Quadratic Weighted Kappa for model  10 is:  0.7898769342381159
Quadratic Weighted Kappa for model  11 is:  0.8075148519584594


In [23]:
# Here we write a sample set of outputs to a dataframe to export for further exploration
t.to_csv('answers.csv', index=False)
learner_dfs[2].to_csv('preds_1.csv', index=False)
learner_dfs[7].to_csv('preds_2.csv', index=False)